### 强化学习

在本章中
* 首先，解释强化学习是什么以及它擅长什么
* 然后，介绍两个在深度强化学习领域最重要的技术
    * 策略梯度
    * 深度Q网络(DQN)
* 包括讨论马尔可夫决策过程(MDP)

我们将使用这些技术来训练一个模型来平衡移动车上的杆子，另一个玩Atari游戏

#### 1. 学习优化奖励(Learning to optimaze rewards)

在强化学习中，智能体在环境(environment)中观察(observation)并且做出决策(action)，随后它会得到奖励(reward)。

目标时区学习如何行动能最大化期望奖励。

简单来说，智能体在环境中行动，并且在实验和错误中去学习最大化它的愉快，最小化它的痛苦。


#### 2. 策略搜索(Policy Search)

被智能体使用去改变它的行为的算法叫做策略。例如：**策略可以是一个把观测当输入，行为当做输出的神经网络**。

![title](policy.png)

**随机策略**：例如一个真空吸尘器，它的奖励是在30分钟内捡起的灰尘的数量，它的策略可以是每秒以概率$P$向前移动，或者以概率$1-P$随机地向左或向右旋转，旋转的角度将是$-R$和$+R$之间的随机角度。由于该策略涉及一些随机性，所以称为随机策略。

**训练方法**
* 通过调整两个策略参数：概率$P$和角度范围$R$。这些参数尝试不同的值，并选择执行最佳的组合，但是当策略空间太大，找到一组好的参数如同大海捞针
* 另一种搜索策略空间的方式是遗传算法：随机创造一个包含100个策略的第一代基因，随后杀死80个糟糕的策略，随后让20个幸存策略繁衍4代。一个后代只是它父辈基因的复制品加上一些随机变异。幸存的策略加上他们的后代共同构成了第二代。继续以这种方式迭代，直到找到一个好的策略。
* 另一种方法是使用优化方法：通过评估奖励关于策略参数的梯度，然后通过跟随梯度向更高的奖励(梯度上升)调整这些参数，这种方法被称为策略梯度(Policy Gradient,PG)

#### 3. OpenAI的介绍

强化学习的一个挑战是，为了训练智能体，首先需要有一个**工作环境**。

训练在现实世界是困难和缓慢的，所以通常需要一个模拟环境，至少需要引导训练。



In [2]:
import gym

In [3]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
env

<TimeLimit<CartPoleEnv<CartPole-v0>>>

In [5]:
obs = env.reset()
obs

array([-0.03321464, -0.03889074,  0.03261536, -0.02290879])

In [6]:
env.render()

True

* 使用$make()$函数创建一个环境，在此例中是CartPole环境。这是一个2D环境，其中推车可以被左右加速，以平衡放置在它上面的平衡杆。
* 使用$reset()$函数初始化，返回第一个观察的结果。观察取决于环境的类型。对于CartPole环境，每个观测是包含四个浮点数的1D Numpy向量，这些浮点数分别代表：
    * 推车的水平位置（0为中心）
    * 推车的速度
    * 杆的角度（0维垂直）
    * 杆的角速度
* 使用$render()$函数显示如图所示的环境

![title](cartpole_env.png)


使用$render()$让图像以一个Numpy数组格式返回，可以将$mode$参数设置为$reg_array$（注意，其他环境可能支持不同的模式）

In [8]:
img = env.render(mode='rgb_array')
img.shape  # height,width,channels(3=RGB)

(400, 600, 3)

CartPole（和其他一些环境）还是会将图像呈现到屏幕上。避免这种情况的唯一方式是使用一个fake X服务器，如XVFB或XDimMy。

可以使用以下命令安装XVFB和启动：

Python :xvfb-run -s "screen 0 1400x900x24" python

或者使用xvfbwrapper包

查询环境的动作空间：

In [9]:
env.action_space

Discrete(2)

Discrete(2)表示可能的动作是整数0和1,表示向左（0）或向右（1）的加速。（其他环境可能有更多的动作甚至是连续的）

因为杆子向右倾斜，让我们向右加速推车：

In [10]:
action = 1  # acclerate right
obs, reward, done, info = env.step(action)

In [11]:
obs

array([-0.03399245,  0.15574866,  0.03215719, -0.3051254 ])

In [14]:
reward

1.0

In [12]:
done

False

In [13]:
info

{}

$step()$表示执行给定的动作并返回四个值：
* $obs$：这是新的观测，小车现在正在往右走（obs[1]>0,注意：当前速度为正，向右为正）。平衡杆仍然向右倾斜(obs[2]>0)，但是它的角速度现在为负(obs[3]<0)，所以它在下一步可能向左倾斜。

* $reward$：在这个环境中，无论做什么，每一步都会得到1.0的奖励，所以游戏的目标就是尽可能长的运行

* $done$：当游戏结束时这个值为True

* $info$：该字典可以在其他环境中提供额外的调试信息。这些数据不应该用于训练（这是作弊）

硬编码一个简单测策略，当杆向左倾斜时加速左边，当杆向右倾斜时加速右边。使用这个策略来获得超过500步的平均回报：

In [19]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle<0 else 1

totals = []  # 获取所有episode的总奖励

for episode in range(500):
    episode_reward = 0  # 获取每个episode的总奖励
    obs = env.reset()
    for step in range(1000):  # 最多1000步，我们不想让它永远运行下去
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        if done:
            break
        env.render()
    totals.append(episode_reward)
        

In [17]:
import numpy as np

In [18]:
np.mean(totals),np.std(totals),np.min(totals),np.max(totals)

(42.02, 8.666925637156465, 24.0, 68.0)

可以看到，即使有500次尝试，这一策略从未使平衡杆在超过68个步骤里保持直立。

并且，推车越来越强烈地左右摆动，知道平衡杆倾斜太多。

#### 4. 神经网络策略(Neural Network Policies)

让我们创建一个神经网络策略。这个神经网络将把观察作为输入，输出要执行的动作，即估计每个动作的概率，然后我们根据估计的概率随机地选择一个动作。

在CartPole环境中，只有两种可能的动作（左或右），所以我们只需要输出一个神经元。他将输出动作0（左）的概率$P$，动作1（右）的概率显然是$1-P$。

例如：如果输出0.7，我们将以70%的概率选择动作0,30%的概率选择动作1.
![title](neural_policy.png)

根据神经网络给出的概率来选择随机的动作，而不是选择最高分数额达动作。

这种方法使智能体在探索新的行为和利用那些已知可行的行动之间找到正确的平衡。

还要注意，在这个特定的环境中过去的动作和观察可以被安全地忽略，因为每个观察都包含环境的完整状态。
* 如果有一些隐藏状态，那么也需要过去的行为和观察
* 另一个例子是当观测是有噪声的，此时需要用过去的观察来估计最可能的当前状态